In [1]:
# IMPORTING PACKAGES
import pandas as pd
import pandas_ta as ta
import matplotlib.pyplot as plt
from termcolor import colored as cl
import yfinance as yf
import math 

In [2]:
def add_donchian_channel_indicator(df):
    df[['dcl', 'dcm', 'dcu']] = df.ta.donchian(lower_length = 10, upper_length = 10)
    df.columns = list(df.columns.str.lower())
    df = df.set_index('date')
    df.index = pd.to_datetime(df.index)
    return df

In [3]:
def generate_signals(row):
    if row['high'] >= row['dcu']:
        return "buy"
    
    elif row['low'] <= row['dcl']:
        return "sell"
    
    else:
        return "hold"

In [4]:
def generate_signals_dochian_channel_strat(df):
    df = add_donchian_channel_indicator(df)
    df['signal'] = df.apply(generate_signals,axis=1)
    return df

In [208]:
def open_long(self,close,date):
    self.open = close
    self.in_position = 1
    self.open_date = date

def close_long(self,close,date):
    self.close = close
    self.equity += self.equity * ((self.close-self.open)/self.open)
    self.in_position = 0
    self.close_date = date
    self.trading_history.append(["Long",[self.open_date,self.open],[self.close_date,self.close]])


def open_short(self,close,date):
    self.open = close
    self.in_position = -1
    self.open_date = date

def close_short(self,close,date):
    self.close = close
    self.equity +=  self.equity * -((self.close-self.open)/self.open)
    self.in_position = 0
    self.close_date = date
    self.trading_history.append(["Short",[self.open_date,self.open],[self.close_date,self.close]])


def simulate_trading_(self):
    for date, row in self.data.iterrows():

        signal = row['signal']
        close = row['close']

        if signal == "buy":

            if self.in_position == 0:
                open_long(self,close,date)

            if self.in_position == -1:
                close_short(self,close,date)

        elif signal == "sell":

            if self.in_position == 0:
                open_short(self,close,date)

            if self.in_position == 1:
                close_long(self,close,date)

                
    
    if self.in_position == 1:
        close_long(self,close,date)

    if self.in_position == -1:
        close_short(self,close,date)


In [209]:
class BaseStrategy:
    def __init__(self, data, initial_equity):
        self.data = data.copy()
        self.equity = initial_equity
        self.no_of_shares = 0
        self.open = 0
        self.close = 0
        self.open_date = ""
        self.close_date = ""
        self.in_position = 0
        self.trading_history = []


    def generate_signals(self):
        raise NotImplementedError("Implement this method in subclasses")
    
    def simulate_trading(self):
        # 1. Generate buy/sell signals (abstract method, implemented in subclasses)
        self.generate_signals()
        simulate_trading_(self)

class dochian_channel(BaseStrategy):
    def generate_signals(self):
        self.data = generate_signals_dochian_channel_strat(self.data)

In [210]:
def get_data(year,ticker):
    start_date = f"{year}-01-01" 
    end_date = f"{year}-12-30"
    interval = "1d"

    df = yf.download(ticker, start=start_date, end=end_date, interval=interval, progress=False).reset_index()

    return df

In [226]:
ticker = "GBPJPY=X"
equity = 1000

strategy = dochian_channel(get_data("2019",ticker),equity)
strategy.simulate_trading()
strategy.equity

1128.9646945496463

In [220]:
strategy.equity

1092.5444111804925

In [199]:
df = pd.DataFrame(strategy.trading_history)

In [200]:
df = pd.DataFrame(strategy.trading_history)
df = pd.DataFrame([(i[0],i[1],j[0],j[1],k) for i,j,k in zip(df[1],df[2],df[0])],columns=['Open Date','Open Price','Close Date','Close Price','Type'])

df['percentage change'] = (df['Open Price'] - df['Close Price'])/df['Open Price']

In [201]:
equity = 500000
for change_ in [j*-1 if i == "Short" else j for i,j in zip(df['Type'],df['percentage change'])]:
    equity = equity + equity * change_

equity

475080.054421586

In [207]:
equity = 500000
equity += equity * df['percentage change'].sum()
equity

485163.97570598806